In [1]:
!mkdir /kaggle/working/train
!ln -s /kaggle/input/sp-cup-train-dataset/train_fake/fake /kaggle/working/train/fake
!ln -s /kaggle/input/sp-cup-train-dataset/train_real/real /kaggle/working/train/real

!mkdir /kaggle/working/test
!ln -s /kaggle/input/sp-cup-fake-validation/valid_fake/fake /kaggle/working/test/fake
!ln -s /kaggle/input/sp-cup-fake-validation/valid_real/real /kaggle/working/test/real

In [2]:
from tensorflow.keras.applications import Xception  
from tensorflow.keras.models import Model  
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D  
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

base_model = Xception(weights='imagenet', include_top=False)  
x = base_model.output  
x = GlobalAveragePooling2D()(x)  
x = Dense(1024, activation='relu')(x)  
predictions = Dense(2, activation='softmax')(x) 

model = Model(inputs=base_model.input, outputs=predictions)  

# Freeze the base model layers  
for layer in base_model.layers:  
    layer.trainable = False

83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  
train_generator = train_datagen.flow_from_directory(  
    '/kaggle/working/train',  # Path to your training data  
    target_size=(299, 299),  
    batch_size=32,  
    class_mode='categorical',  
    subset='training')  
validation_generator = train_datagen.flow_from_directory(  
    '/kaggle/working/train',  
    target_size=(299, 299),  
    batch_size=32,  
    class_mode='categorical',  
    subset='validation')

Found 209728 images belonging to 2 classes.
Found 52432 images belonging to 2 classes.


In [5]:
model.fit(train_generator, validation_data=validation_generator, epochs=10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


6554/6554 ━━━━━━━━━━━━━━━━━━━━ 2735s 415ms/step - accuracy: 0.8360 - loss: 0.4289 - val_accuracy: 0.8387 - val_loss: 0.3936
Epoch 2/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1821s 278ms/step - accuracy: 0.8407 - loss: 0.3894 - val_accuracy: 0.8437 - val_loss: 0.3828
Epoch 3/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1628s 248ms/step - accuracy: 0.8479 - loss: 0.3678 - val_accuracy: 0.8471 - val_loss: 0.3697
Epoch 4/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1556s 237ms/step - accuracy: 0.8527 - loss: 0.3496 - val_accuracy: 0.8530 - val_loss: 0.3498
Epoch 5/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1577s 240ms/step - accuracy: 0.8578 - loss: 0.3375 - val_accuracy: 0.8541 - val_loss: 0.3473
Epoch 6/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1582s 241ms/step - accuracy: 0.8634 - loss: 0.3208 - val_accuracy: 0.8578 - val_loss: 0.3493
Epoch 7/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1558s 238ms/step - accuracy: 0.8673 - loss: 0.3105 - val_accuracy: 0.8592 - val_loss: 0.3319
Epoch 8/10
6554/6554 ━━━━━━━━━━━━━━━━━━━━ 1550s 236ms/step - accur

In [6]:
model.save('xception_model.h5')

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator  

# Create a test data generator  
test_datagen = ImageDataGenerator(rescale=1./255)  

test_generator = test_datagen.flow_from_directory(  
    '/kaggle/working/test',  # Path to your test dataset  
    target_size=(299, 299),  # Resize the images to match the input size  
    batch_size=32,  
    class_mode='categorical',  # Set to 'categorical' for multi-class  
    shuffle=False  # Important: Do not shuffle the test data  
)

Found 3072 images belonging to 2 classes.


In [8]:
# Evaluate the model on the test data  
test_loss, test_accuracy = model.evaluate(test_generator)

96/96 ━━━━━━━━━━━━━━━━━━━━ 28s 286ms/step - accuracy: 0.8374 - loss: 0.4818
